## Observations and Insights 

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import sem

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
mouse_df=pd.merge(mouse_metadata, study_results, on="Mouse ID")

In [4]:
# Checking the number of mice in the DataFrame.
unique_mice=mouse_df["Mouse ID"].unique()
total_mice=0
for mouse in unique_mice:
    total_mice+=1
print(f"There are {total_mice} mice in this dataset.")
print(f"There are {mouse_df.shape[0]} rows in this dataset.")

#also could possibly do len(unique_mice)

There are 249 mice in this dataset.
There are 1893 rows in this dataset.


In [5]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 

In [6]:
# Optional: Get all the data for the duplicate mouse ID. 
extra_mouse=mouse_df[mouse_df.duplicated()]
extra_mouse

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
909,g989,Propriva,Female,21,26,0,45.0,0


In [7]:
# Create a clean DataFrame by dropping the duplicate mouse.
clean_mouse_df=mouse_df.drop_duplicates(keep="first")

In [8]:
# Checking the number of mice in the clean DataFrame.
unique_cleanmouse=clean_mouse_df["Mouse ID"].unique()
total_cleanmouse=0
for mouse in unique_cleanmouse:
    total_cleanmouse+=1
print(f"There are {total_cleanmouse} mice in this dataset.")
print(f"There are {clean_mouse_df.shape[0]} rows in this dataset.")

There are 249 mice in this dataset.
There are 1892 rows in this dataset.


In [9]:
clean_mouse_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


## Summary Statistics

In [50]:
#Note: I talked to Andrew about my homework and he showed me the alternative (probably better) ways to solve this problem. 
# I'll show all three of them here so that I'm not plagarizing his methods but I still get to practice them. Below is the
#first way I solved the problem, and the next two cells are his methods.

#METHOD 1: LOOP THROUGH THE DATA AND BUILD A DATAFRAME

drug_means=[]
drug_medians=[]
drug_modes=[]
drug_sds=[]
drug_SEMs=[]

#----get the drugs
drugs=clean_mouse_df["Drug Regimen"].unique()

#----loop through the data and populate a list of summary statistics for each drug
for drug in drugs:
    drug_means.append (np.mean((clean_mouse_df.loc[clean_mouse_df["Drug Regimen"]==drug, "Tumor Volume (mm3)"]))), #means
    drug_medians.append(np.median((clean_mouse_df.loc[clean_mouse_df["Drug Regimen"]==drug, "Tumor Volume (mm3)"]))) #medians
    drug_modes.append(st.mode((clean_mouse_df.loc[clean_mouse_df["Drug Regimen"]==drug, "Tumor Volume (mm3)"]))[0]) #modes
    drug_sds.append((clean_mouse_df.loc[clean_mouse_df["Drug Regimen"]==drug, "Tumor Volume (mm3)"]).std()) #sds
    drug_SEMs.append((clean_mouse_df.loc[clean_mouse_df["Drug Regimen"]==drug, "Tumor Volume (mm3)"]).sem())
  

In [51]:
drug_summary_stats =pd.DataFrame({
    "Drug":(drug for drug in drugs),
    "Mean":[value for value in drug_means],
    "Median":[value for value in drug_medians],
    "Mode":[value for value in drug_medians],
    "Standard Deviation":[value for value in drug_sds],
    "Variance":[value**2 for value in drug_sds],
    "SEM ":[value for value in drug_SEMs]})

In [52]:
drug_summary_stats

,Drug,Mean,Median,Mode,Standard Deviation,Variance,SEM
0,Ramicane,40.216745,40.673236,40.673236,4.846308,23.486704,0.320955
1,Capomulin,40.675741,41.557809,41.557809,4.994774,24.947764,0.329346
2,Infubinol,52.884795,51.820584,51.820584,6.567243,43.128684,0.492236
3,Placebo,54.033581,52.288934,52.288934,7.821003,61.168083,0.581331
4,Ceftamin,52.591172,51.776157,51.776157,6.268188,39.290177,0.469821
5,Stelasyn,54.233149,52.431737,52.431737,7.710419,59.450562,0.573111
6,Zoniferol,53.236507,51.818479,51.818479,6.966589,48.533355,0.516398
7,Ketapril,55.235638,53.698743,53.698743,8.279709,68.553577,0.603860
8,Propriva,52.368318,50.909965,50.909965,6.502160,42.278090,0.514041
9,Naftisol,54.331565,52.509285,52.509285,8.134708,66.173479,0.596466


In [56]:
#Get summary statistics using the groupby method:
#Note: I didn't get the mode to work using this method. I'm sure that I could, but I wanted to continue on with the assignment.
#without further delay.

grouped=clean_mouse_df.groupby(["Drug Regimen"])
means=grouped.mean()["Tumor Volume (mm3)"]
median=grouped.median()["Tumor Volume (mm3)"]
standard_dev=grouped.std()["Tumor Volume (mm3)"]
sem=grouped.sem()["Tumor Volume (mm3)"]
drug_summary_stats2 =pd.DataFrame({
    "Mean":(means),
    "Median":(median),
    "SD":(standard_dev),
    "VAR":(standard_dev**2),
    "SEM":(sem)})

In [57]:
drug_summary_stats2

,Mean,Median,SD,VAR,SEM
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,4.994774,24.947764,0.329346
Ceftamin,52.591172,51.776157,6.268188,39.290177,0.469821
Infubinol,52.884795,51.820584,6.567243,43.128684,0.492236
Ketapril,55.235638,53.698743,8.279709,68.553577,0.603860
Naftisol,54.331565,52.509285,8.134708,66.173479,0.596466
Placebo,54.033581,52.288934,7.821003,61.168083,0.581331
Propriva,52.368318,50.909965,6.502160,42.278090,0.514041
Ramicane,40.216745,40.673236,4.846308,23.486704,0.320955
Stelasyn,54.233149,52.431737,7.710419,59.450562,0.573111


In [27]:
# get summary statistics using the .agg method. This is basically the same code that Andrew showed me,
#but I still watned to practice it. To make up for it I watched a video on how this works, if that helps? 
#I also still coudln't get the mode to work here, but wanted to move on.

drug_summary_stats3=grouped.agg({"Tumor Volume (mm3)":["mean","median","var","std","sem"]})
drug_summary_stats3

Tumor Volume (mm3)                                          
                           mean     median        var       std       sem
Drug Regimen                                                             
Capomulin             40.675741  41.557809  24.947764  4.994774  0.329346
Ceftamin              52.591172  51.776157  39.290177  6.268188  0.469821
Infubinol             52.884795  51.820584  43.128684  6.567243  0.492236
Ketapril              55.235638  53.698743  68.553577  8.279709  0.603860
Naftisol              54.331565  52.509285  66.173479  8.134708  0.596466
Placebo               54.033581  52.288934  61.168083  7.821003  0.581331
Propriva              52.368318  50.909965  42.278090  6.502160  0.514041
Ramicane              40.216745  40.673236  23.486704  4.846308  0.320955
Stelasyn              54.233149  52.431737  59.450562  7.710419  0.573111
Zoniferol             53.236507  51.818479  48.533355  6.966589  0.516398

## Bar Plots

In [9]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pandas. 

In [10]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pyplot.

## Pie Plots

In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas

In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot

## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the most promising treatment regimens. Calculate the IQR and quantitatively determine if there are any potential outliers. 


In [14]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest

## Line and Scatter Plots

In [15]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [16]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen



## Correlation and Regression

In [17]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
